In [1]:
!pip install spark-nlp==1.7.3

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StringType, DataType
import pandas as pd
from pyspark.sql.functions import udf,col
from pyspark.sql import functions as F
pd.set_option('max_colwidth', 800)
import matplotlib
from pyspark.sql.types import TimestampType, StructType, StructField, StringType, FloatType, IntegerType, LongType
from pyspark.sql import types as T
from datetime import datetime

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName(" Sparkify Localy") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.8.2") \
    .getOrCreate()
spark

In [ ]:
# get parameters
spark.sparkContext.getConf().getAll()

## Log Data Default Schema

In [ ]:
log_data_path = "./log_data/*.json"
df_log = spark.read.json(log_data_path)
df_log
print('DataFrame rows: %d' % df_log.count())
df_log.printSchema()
print('DataFrame schema: %s' % df_log)
df_log.show(2)

## Log Data My Schema

In [ ]:
log_data_path = "./log_data/*.json"
log_schema = StructType([
    StructField("artist", StringType()),
    StructField("auth", StringType()),
    StructField("firstName", StringType()),
    StructField("gender", StringType()),
    StructField("itemInSession", IntegerType()),
    StructField("lastName", StringType()),
    StructField("length", FloatType()),    
    StructField("level", StringType()),
    StructField("location", StringType()),
    StructField("method", StringType()),
    StructField("page", StringType()),
    StructField("registration", FloatType()),
    StructField("sessionId", StringType()),
    StructField("song", StringType()),
    StructField("status", IntegerType()),
    StructField("ts", LongType()),
    StructField("userAgent", StringType()),
    StructField("userId", IntegerType())
])
df_logS = spark.read.json(log_data_path, schema=log_schema)
df_logS
print('DataFrame rows: %d' % df_logS.count())
df_logS.printSchema()
print('DataFrame schema: %s' % df_logS)
df_logS.show(2)

## Song Data My schema

In [ ]:
song_data_path = "./song_data/A/A/A/*.json"
song_schema = StructType([
    StructField("num_songs", IntegerType()),
    StructField("artist_id", StringType()),
    StructField("artist_latitude", FloatType()),
    StructField("artist_longitude", FloatType()),
    StructField("artist_location", StringType()),
    StructField("artist_name", StringType()),
    StructField("song_id", StringType()),
    StructField("title", StringType()),
    StructField("duration", FloatType()),
    StructField("year", IntegerType())   
])
df_song = spark.read.json(song_data_path, schema = song_schema)
print('DataFrame rows: %d' % df_song.count())
df_song.printSchema()
print('DataFrame schema: %s' % df_song)
df_song.show(3)